In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# вместо LinearRegression будем использовать Ridge
# это численно более стабильная модификация обычной линейной регрессии
from sklearn.linear_model import Ridge
# измерять качество предсказаний будем с помощью стандартной MSE
from sklearn.metrics import mean_squared_error
# для нормализации данных будем использовать MinMixScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def visualize_coefficients(coefs, feature_names, top_n):
    """Функция для визуализации коэффициентов линейной регрессии.

    Параметры:
        coefs: коэффициенты модели (model.coef_).
        feature_names: названия признаков (X_train.columns).
        top_n: вывести top_n самых положительных и top_n самых отрицательных признаков.
    """
    feature_names = np.array(feature_names)
    if top_n * 2 > len(coefs):
        n_pos = len(coefs) // 2
        n_neg = len(coefs) - n_pos
    else:
        n_pos, n_neg = top_n, top_n
    # нам нужно найти индексы top_n наибольших и top_n наименьших коэффициентов
    min_coef_idxs = np.argsort(coefs)[:n_neg]
    max_coef_idxs = np.argsort(coefs)[len(coefs) - n_pos:]
    # соответствующие имена фичей
    top_feature_names = np.concatenate((feature_names[min_coef_idxs], feature_names[max_coef_idxs])) 
    # отобразим на bar-графике
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.bar(np.arange(n_neg), coefs[min_coef_idxs], color=sns.xkcd_rgb["mauve"], hatch="/")
    ax.bar(np.arange(n_neg, n_neg + n_pos), coefs[max_coef_idxs], color=sns.xkcd_rgb["teal"], hatch="\\")
    ax.set_xticks(np.arange(0, n_neg + n_pos))
    ax.set_xticklabels(top_feature_names, rotation=45, ha="right", fontsize=14)
    plt.show()

In [ ]:
def visualize_preds(y_true, y_pred, n_hours=336):
    """Функция для визуализации таргета и предсказаний.

    Параметры:
        y_true: правильные ответы.
        y_pred: предсказания модели.
        n_hours: вывести на график заданное число точек с конца. По умолчанию 2 недели.

    """
    fig, ax = plt.subplots(figsize=(21, 9))
    index = y_true[-n_hours:].index
    ax.plot(index, y_true[-n_hours:], label="y_true")
    ax.plot(index, y_pred[-n_hours:], label="y_pred")
    ax.legend()

    delta_len = len(y_true) - len(y_pred)
    val_mse = mean_squared_error(y_true[delta_len:], y_pred)
    ax.set_title(f"Validation MSE: {val_mse}")
    plt.show()

# Загрузка данных

In [ ]:
# parse_dates позволяет сразу распарсить даты в данных
# с помощью index_col указываем что будем использовать колонку с датой в качестве индекса датафрейма
train_df = pd.read_csv("indian-metro-train.csv", parse_dates=["date_time"], index_col="date_time")
val_df = pd.read_csv("indian-metro-val.csv", parse_dates=["date_time"], index_col="date_time")

In [ ]:
train_df.head()

In [ ]:
val_df.head()

Как видно, датафрейм содержит измерения количества людей в метро. Для каждого измерения доступна его дата и время (с разрешением до часа), а также набор признаков, описывающих ситуацию на момент измерения:


*   is_holiday: название праздника, если он проводится в этот день
*   air_pollution_index: индекс качества воздуха
*   humidity: относительная влажность в градусах Цельсия
*   wind_speed: скорость ветра в миль/ч
*   wind_direction: направление ветра в градусах
*   visibility_in_miles: видимость в милях
*   temperature: температура в Кельвинах
*   rain_p_h: количество выпавших осадков в мм за час
*   clouds_all
*   weather_type
*   weather_description: 
*   traffic_volume: количество людей, целевая переменная



Будем решать задачу предсказания traffic_volume по остальным признакам. Оценивать качество будем с помощью MSE на валидационной выборке. На семинаре также оценим ваши лучшие модели на отдельной тестовой выборке.

# Trainee

Попробуем обучить самую простую модель. Датасет содержит 3 нечисловых признака, с которыми придется отдельно придумать, что делать, поэтому просто исключим их (не забыв исключить и целевую переменную):

In [ ]:
# рекомендую оформлять свой код подготовки данных в функции похожего вида
def prepare_data_trainee(train_df, val_df):
    X_train = train_df.drop(["is_holiday", "weather_type", "weather_description", "traffic_volume"], axis=1)
    X_val = val_df.drop(["is_holiday", "weather_type", "weather_description", "traffic_volume"], axis=1)
    # сохраним названия признаков для отрисовки графика
    feature_names = X_train.columns.tolist()

    y_train = train_df["traffic_volume"]
    y_val = val_df["traffic_volume"]

    # нормализуем данные для линейной регрессии
    scl = MinMaxScaler()
    X_train = scl.fit_transform(X_train)
    # !Важно: для train применяем метод .fit_transform(), для валидации и/или тест - .transform()
    X_val = scl.transform(X_val)
    return X_train, y_train, X_val, y_val, feature_names

In [ ]:
def fit_ridge(train_df, val_df, prepare_func, alpha=1.0):
    X_train, y_train, X_val, y_val, feature_names = prepare_func(train_df, val_df)
    model = Ridge(alpha=alpha, random_state=14300631)
    model.fit(X_train, y_train)

    train_predictions = model.predict(X_train)
    val_predictions = model.predict(X_val)

    train_mse = mean_squared_error(y_train, train_predictions)
    val_mse = mean_squared_error(y_val, val_predictions)

    print(f"Train MSE: {train_mse}")
    print(f"Validation MSE: {val_mse}")
    return {
        "model": model,
        "train_preds": train_predictions,
        "val_preds": val_predictions,
        "features": feature_names,
    }

In [ ]:
result_trainee = fit_ridge(train_df, val_df, prepare_data_trainee)

Модель успешно обучилась. Большой разницы между ошибкой на обучении и валидации не видно, значит как минимум модель пока что не переобучается :)

Давайте посмотрим как выглядит целевая переменная и предсказании на графике:

In [ ]:
visualize_preds(val_df["traffic_volume"], result_trainee["val_preds"])

Модель явно слишком слабая - ей не хватает признаков, чтобы ухватить зависимость сложнее, чем предсказание около среднего.

Посмотрим также на влияние признаков:

In [ ]:
visualize_coefficients(result_trainee["model"].coef_, result_trainee["features"], 15)

Забавно. Чем выше температура на улице - тем больше людей в метро по мнению модели, а чем больше осадков выпадало - тем меньше :)

# Junior

In [ ]:
def add_time_features(df):
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["day"] = df.index.day
    df["hour"] = df.index.hour
    return df


def prepare_data_junior(train_df, val_df):
    X_train = train_df.drop(["is_holiday", "weather_type", "weather_description", "traffic_volume"], axis=1)
    X_val = val_df.drop(["is_holiday", "weather_type", "weather_description", "traffic_volume"], axis=1)

    # добавляем временные признаки
    X_train = add_time_features(X_train)
    X_val = add_time_features(X_val)
    # сохраним названия признаков для отрисовки графика
    feature_names = X_train.columns.tolist()

    y_train = train_df["traffic_volume"]
    y_val = val_df["traffic_volume"]

    # нормализуем данные для линейной регрессии
    scl = MinMaxScaler()
    X_train = scl.fit_transform(X_train)
    # !Важно: для train применяем метод .fit_transform(), для валидации и/или тест - .transform()
    X_val = scl.transform(X_val)
    return X_train, y_train, X_val, y_val, feature_names

In [ ]:
result_junior = fit_ridge(train_df, val_df, prepare_data_junior)

In [ ]:
visualize_preds(val_df["traffic_volume"], result_junior["val_preds"])

In [ ]:
visualize_coefficients(result_junior["model"].coef_, result_junior["features"], 15)

# Middle

In [ ]:
from sklearn.preprocessing import OneHotEncoder


def additional_time_features(df):
    df['is_night'] = df['hour'].apply(lambda x: 0 <= x <= 4).astype('int8')
    df['is_morning_peak'] = df['hour'].apply(lambda x: 6 <= x <= 8).astype('int8')
    df['is_evening_peak'] = df['hour'].apply(lambda x: 15 <= x <= 17).astype('int8')
    df['weekday'] = df.index.weekday
    df['weekend'] = df['weekday'].apply(lambda x: x in (5, 6)).astype('int8')
    return df


def is_any_holiday(df):
    df["any_holiday"] = df["is_holiday"].notna()
    df = df.drop("is_holiday", axis=1)
    return df


def dummy_encoding(df, encoder=None):
    ohe_features = ["weather_type", "weather_description"]
    for obj_feature in ohe_features:
        df[obj_feature] = df[obj_feature].str.lower().str.replace(" ", "_")

    if encoder is None:
        encoder = OneHotEncoder(sparse=False, dtype=np.int8, handle_unknown='ignore')
        encoded_features = encoder.fit_transform(df[ohe_features])
    else:
        encoded_features = encoder.transform(df[ohe_features])
    
    ohe_feature_names = encoder.get_feature_names_out(ohe_features)
    df[ohe_feature_names] = encoded_features
    df = df.drop(ohe_features, axis=1)
    return df, encoder


def prepare_data_middle(train_df, val_df):
    X_train = train_df.drop(["traffic_volume"], axis=1)
    X_val = val_df.drop(["traffic_volume"], axis=1)

    # добавляем временные признаки
    X_train = add_time_features(X_train)
    X_val = add_time_features(X_val)

    # дополнительные признаки периодов дня
    X_train = additional_time_features(X_train)
    X_val = additional_time_features(X_val)

    # работаем с нечисленными признаками
    # сделаем бинарный признак праздник / не праздник
    X_train = is_any_holiday(X_train)
    X_val = is_any_holiday(X_val)

    # dummy-кодирование для признаков weather_type, weather_description
    X_train, encoder = dummy_encoding(X_train)
    X_val, _ = dummy_encoding(X_val, encoder)

    # сохраним названия признаков для отрисовки графика
    feature_names = X_train.columns.tolist()

    y_train = train_df["traffic_volume"]
    y_val = val_df["traffic_volume"]

    # нормализуем данные для линейной регрессии
    scl = MinMaxScaler()
    X_train = scl.fit_transform(X_train)
    # !Важно: для train применяем метод .fit_transform(), для валидации и/или тест - .transform()
    X_val = scl.transform(X_val)
    return X_train, y_train, X_val, y_val, feature_names

In [ ]:
result_middle = fit_ridge(train_df, val_df, prepare_data_middle)

In [ ]:
visualize_preds(val_df["traffic_volume"], result_middle["val_preds"])

In [ ]:
visualize_coefficients(result_middle["model"].coef_, result_middle["features"], 15)

# Senior

In [ ]:
def add_lags(df, n_lags):
    df = df.copy()
    for lag_idx in range(1, n_lags + 1):
        df[f'lag_{lag_idx}'] = df['traffic_volume'].shift(lag_idx)
    df = df.dropna()
    return df


def cycle_features(df):
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    
    df['cos_weekday'] = np.cos(2 * np.pi * df['weekday'] / 7)
    df['sin_weekday'] = np.sin(2 * np.pi * df['weekday'] / 7)
    
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    
    df['cos_wind_direction'] = np.cos(2 * np.pi * df['wind_direction'] / 360)
    df['sin_wind_direction'] = np.sin(2 * np.pi * df['wind_direction'] / 360)

    df = df.drop(["hour", "weekday", "month", "wind_direction"], axis=1)
    return df


def additional_weather_features(df):
    df['not_bad_weather'] = df['weather_type_clear'] | df['weather_type_clouds']
    df['bad_weather'] = df['weather_type_rain'] | df['weather_type_snow'] | df['weather_type_thunderstorm'] | df['weather_type_squall']
    df['doubtful_weather'] = df['weather_type_mist'] | df['weather_type_drizzle'] | df['weather_type_haze'] | df['weather_type_fog']
    return df


def prepare_data_senior(train_df, val_df):
    # добавляем лаги целевой переменной
    train_df = add_lags(train_df, n_lags=12)
    val_df = add_lags(val_df, n_lags=12)

    X_train = train_df.drop(["traffic_volume"], axis=1)
    X_val = val_df.drop(["traffic_volume"], axis=1)

    # добавляем временные признаки
    X_train = add_time_features(X_train)
    X_val = add_time_features(X_val)

    # дополнительные признаки периодов дня
    X_train = additional_time_features(X_train)
    X_val = additional_time_features(X_val)

    # циклическое кодирование
    X_train = cycle_features(X_train)
    X_val = cycle_features(X_val)

    # работаем с нечисленными признаками
    # сделаем бинарный признак праздник / не праздник
    X_train = is_any_holiday(X_train)
    X_val = is_any_holiday(X_val)

    # dummy-кодирование для признаков weather_type, weather_description
    X_train, encoder = dummy_encoding(X_train)
    X_val, _ = dummy_encoding(X_val, encoder)

    # дополнительные признаки погоды
    X_train = additional_weather_features(X_train)
    X_val = additional_weather_features(X_val)

    # сохраним названия признаков для отрисовки графика
    feature_names = X_train.columns.tolist()

    y_train = train_df["traffic_volume"]
    y_val = val_df["traffic_volume"]

    # нормализуем данные для линейной регрессии
    scl = MinMaxScaler()
    X_train = scl.fit_transform(X_train)
    # !Важно: для train применяем метод .fit_transform(), для валидации и/или тест - .transform()
    X_val = scl.transform(X_val)
    return X_train, y_train, X_val, y_val, feature_names

In [ ]:
result_senior = fit_ridge(train_df, val_df, prepare_data_senior)

In [ ]:
visualize_preds(val_df["traffic_volume"], result_senior["val_preds"])

In [ ]:
visualize_coefficients(result_middle["model"].coef_, result_senior["features"], 20)

# Результаты

In [ ]:
results = pd.DataFrame({
    "level": ["trainee", "junior", "middle", "senior"],
    "val_mse": [3947564.7226819396, 3487925.9713073457, 1238385.1927409833, 452914.9664840798]
})
results["val_rmse"] = np.sqrt(results["val_mse"])
results["rel_improvement"] = results["val_rmse"].pct_change() * 100

In [ ]:
results

# Тест

In [ ]:
test_df = pd.read_csv("indian-metro-test.csv", parse_dates=["date_time"], index_col="date_time")

In [ ]:
def prepare_data(train_df, val_df, test_df):
    # добавляем лаги целевой переменной
    train_df = add_lags(train_df, n_lags=12)
    val_df = add_lags(val_df, n_lags=12)
    test_df = add_lags(test_df, n_lags=12)

    X_train = train_df.drop(["traffic_volume"], axis=1)
    X_val = val_df.drop(["traffic_volume"], axis=1)
    X_test = test_df.drop(["traffic_volume"], axis=1)

    # добавляем временные признаки
    X_train = add_time_features(X_train)
    X_val = add_time_features(X_val)
    X_test = add_time_features(X_test)

    # дополнительные признаки периодов дня
    X_train = additional_time_features(X_train)
    X_val = additional_time_features(X_val)
    X_test = additional_time_features(X_test)

    # циклическое кодирование
    X_train = cycle_features(X_train)
    X_val = cycle_features(X_val)
    X_test = cycle_features(X_test)

    # работаем с нечисленными признаками
    # сделаем бинарный признак праздник / не праздник
    X_train = is_any_holiday(X_train)
    X_val = is_any_holiday(X_val)
    X_test = is_any_holiday(X_test)

    # dummy-кодирование для признаков weather_type, weather_description
    X_train, encoder = dummy_encoding(X_train)
    X_val, _ = dummy_encoding(X_val, encoder)
    X_test, _ = dummy_encoding(X_test, encoder)

    # дополнительные признаки погоды
    X_train = additional_weather_features(X_train)
    X_val = additional_weather_features(X_val)
    X_test = additional_weather_features(X_test)

    # сохраним названия признаков для отрисовки графика
    feature_names = X_train.columns.tolist()

    y_train = train_df["traffic_volume"]
    y_val = val_df["traffic_volume"]
    y_test = test_df["traffic_volume"]

    # нормализуем данные для линейной регрессии
    scl = MinMaxScaler()
    X_train = scl.fit_transform(X_train)
    # !Важно: для train применяем метод .fit_transform(), для валидации и/или тест - .transform()
    X_val = scl.transform(X_val)
    X_test = scl.transform(X_test)
    return X_train, y_train, X_val, y_val, X_test, y_test, feature_names

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, feature_names = prepare_data(train_df, val_df, test_df)

In [ ]:
model = Ridge(alpha=1.0, random_state=14300631)
model.fit(X_train, y_train)

train_predictions = model.predict(X_train)
val_predictions = model.predict(X_val)
test_predictions = model.predict(X_test)

train_mse = mean_squared_error(y_train, train_predictions)
val_mse = mean_squared_error(y_val, val_predictions)
test_mse = mean_squared_error(y_test, test_predictions)

print(f"Train MSE: {train_mse}")
print(f"Validation MSE: {val_mse}")
print(f"Test MSE: {test_mse}")

In [ ]:
visualize_preds(test_df["traffic_volume"], test_predictions)